In [ ]:
# To start with, we use the default stack
!zenml init

# We also need to connect to a remote ZenML Instance
!zenml connect --url https://1cf18d95-zenml.cloudinfra.zenml.io

# Now we set the stack
!zenml stack set local-mlflow-stack

In [ ]:
# Do the imports at the top

import random
from zenml import ExternalArtifact, pipeline 
from zenml.client import Client
from zenml.logger import get_logger
from uuid import UUID

import os
from typing import Optional, List

from zenml import pipeline

from pipelines import feature_engineering

from steps import (
    data_loader,
    data_preprocessor,
    data_splitter,
    model_evaluator,
    model_trainer,
    inference_predict,
    inference_preprocessor
)

logger = get_logger(__name__)

client = Client()

# Start local with a simple training pipeline

Below you can see what the pipeline looks like. We will start by running this locally on the default-stack. This means the data between the steps is stored locally and the compute is also local.

<img src="_assets/default_stack.png" alt="Drawing" style="width: 800px;"/>

In [ ]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from zenml import step
from zenml.logger import get_logger

logger = get_logger(__name__)

# Here is what one of the steps in the pipeline looks like. Simple python function that just needs the `@step` decorator.

@step
def data_loader() -> pd.DataFrame:
    """Dataset reader step."""
    dataset = load_breast_cancer(as_frame=True)
    inference_size = int(len(dataset.target) * 0.05)
    dataset: pd.DataFrame = dataset.frame
    dataset.reset_index(drop=True, inplace=True)
    logger.info(f"Dataset with {len(dataset)} records loaded!")

    return dataset


In [ ]:
# Here's an example of what this function returns

data_loader()

In [ ]:
from zenml import pipeline

@pipeline
def training(
    train_dataset_id: Optional[UUID] = None,
    test_dataset_id: Optional[UUID] = None,
    min_train_accuracy: float = 0.0,
    min_test_accuracy: float = 0.0,
):
    """Model training pipeline."""
    # Execute Feature Engineering Pipeline
    dataset_trn, dataset_tst = feature_engineering()

    model = model_trainer(
        dataset_trn=dataset_trn,
    )

    model_evaluator(
        model=model,
        dataset_trn=dataset_trn,
        dataset_tst=dataset_tst,
        min_train_accuracy=min_train_accuracy,
        min_test_accuracy=min_test_accuracy,
    )


In [ ]:
pipeline_args = {
    "enable_cache": True, 
    "model_version": {
        "name": "breast_cancer_classifier",
        "description": "Classification of Breast Cancer Dataset.",
        "tags": ["classification", "sklearn"]
    }
}

# Model Version config
fe_t_configured = training.with_options(**pipeline_args)

In [ ]:
fe_t_configured()

# Let's outsource some compute to Sagemaker!

Let's farm some compute to AWS with a training job with a certain number of CPUs and Memory. This can easily be done without and changes to the actual implementation of the pipeline. 














<img src="_assets/local_sagmaker_so_stack.png" alt="Drawing" style="width: 800px;"/>

In [ ]:
# This pip installs the requirements locally
!zenml integration install aws s3 mlflow -y

# This changes the active stack
!zenml stack set local-sagemaker-step-operator-stack

In [ ]:
from zenml.config import DockerSettings

# The actual code will stay the same, all that needs to be done is some configuration

step_args = {}

# We configure which step operator should be used
step_args["step_operator"] = "sagemaker-eu"

# M5 Large is what we need for this big data!
step_args["settings"] = {"step_operator.sagemaker": {"estimator_args": {"instance_type" : "ml.m5.large"}}}

# Update the step. We could also do this in YAML
model_trainer = model_trainer.with_options(**step_args)

docker_settings = DockerSettings(
    requirements=[
        "pyarrow",
        "scikit-learn==1.1.1"
    ],
)

pipeline_args = {
    "enable_cache": True, 
    "model_version": {
        "name": "breast_cancer_classifier",
        "description": "Classification of Breast Cancer Dataset.",
        "tags": ["classification", "sklearn"]
    },
    "settings": {"docker": docker_settings}
}

fe_t_configured = training.with_options(**pipeline_args)

In [ ]:
# Lets run the pipeline
fe_t_configured()

# Switch to full Sagemaker Stack

Just one command will allow you to switch the full code execution over to sagemaker. No Sagemaker domain knowledge necessary. No setup of VMs or Kubernetes clusters necessary. No maintenance of any infrastructure either.

![Sagemaker local stack](_assets/sagemaker_stack.png)


In [ ]:
# Finally, this is all that needs to be done to fully switch the code to be run fully on sagemaker

!zenml stack set sagemaker-stack

In [ ]:
fe_t_configured()